Utilizando o template proposto, notas de aulas e os livros disponibilizados faça uma análise das palavras mais significativas (por frequência), eliminando ao máximo preposições, artigos, pronomes e outras classes de palavras que não contribuírem na sua análise.

Ao final, analise os resultados, como por exemplo: O que é esperado e o que não é? O que a frequência dos termos podem dizer sobre o livro? Como seu trabalho poderia evoluir no futuro?

Utilizando o template proposto, notas de aulas e os livros disponibilizados
    faça uma análise das palavras mais significativas (por frequência),
    eliminando ao máximo preposições, artigos, pronomes e outras classes
    de palavras que não contribuírem na sua análise.
Ao final, analise os resultados, como, por exemplo:
    O que é esperado e o que não é?
    O que a frequência dos termos podem dizer sobre o livro?
    Como seu trabalho poderia evoluir no futuro?

In [1]:
from pyspark.sql.functions import split, explode, col, lower, regexp_extract
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession
from pandas import DataFrame


def build_spark_session(app_name: str) -> SparkSession:
    """
    Build a SparkSession object.
    :return: SparkSession object
    """
    return SparkSession.builder.appName(app_name).getOrCreate()


def read_text_file(spark: SparkSession, file_name: str) -> DataFrame:
    """
    Read a text file and return a DataFrame.
    :param spark: SparkSession object
    :param file_name: File name
    :return: DataFrame
    """
    return spark.read.text(file_name)


def split_lines(df: DataFrame) -> DataFrame:
    """
    Split lines into words.
    :param df: DataFrame
    :return: DataFrame
    """
    return df.select(split(df.value, " ").alias("line"))


def split_words(df: DataFrame) -> DataFrame:
    """
    Split lines into words.
    :param df: DataFrame
    :return: DataFrame
    """
    return df.select(explode(col("line")).alias("word"))


def lower_words(df: DataFrame) -> DataFrame:
    """
    Lower words.
    :param df: DataFrame
    :return: DataFrame
    """
    return df.select(lower(col("word")).alias("lowered_word"))


def clean_words(df: DataFrame) -> DataFrame:
    """
    Clean words.
    :param df: DataFrame
    :return: DataFrame
    """
    return df.select(regexp_extract(col("lowered_word"), "[a-z]+", 0).alias("cleaned_word"))


def remove_empty_words(df: DataFrame) -> DataFrame:
    """
    Remove empty words.
    :param df: DataFrame
    :return: DataFrame
    """
    return df.filter(col("cleaned_word") != "").select(col("cleaned_word").alias("word"))


def remove_stop_words(df: DataFrame) -> DataFrame:
    """
    Remove stop words.
    :param df: DataFrame
    :return: DataFrame
    """
    stop_words_remover = StopWordsRemover(inputCol="word", outputCol="meaningful")
    words_list_df = df.select(split(col("word"), " ").alias("word"))
    return stop_words_remover.transform(words_list_df).select((col("meaningful")[0]).alias("meaningful"))


def remove_nulls(df: DataFrame) -> DataFrame:
    """
    Remove nulls.
    :param df: DataFrame
    :return: DataFrame
    """
    return df.filter(col("meaningful") != "").where(col("meaningful") != "null")


def count_words(df: DataFrame) -> DataFrame:
    """
    Count words.
    :param df: DataFrame
    :return: DataFrame
    """
    return df.groupBy("meaningful").count().orderBy(col("count").desc())


def meaningful_words_counter(file_name: str) -> DataFrame:
    """
    Count meaningful words.
    :param file_name: File name
    :return: DataFrame
    """
    spark = build_spark_session("Meaningful Words Counter")
    df = read_text_file(spark, file_name)
    df = split_lines(df)
    df = split_words(df)
    df = lower_words(df)
    df = clean_words(df)
    df = remove_empty_words(df)
    df = remove_stop_words(df)
    df = remove_nulls(df)
    df = count_words(df)
    return df

In [3]:
books_filepath = "books/{}.txt"
books = [
    books_filepath.format(book_name) for book_name in
    ["frankenstein", "moby-dick", "pride-and-prejudice",
     "the-adventures-of-sherlock-holmes", "the-romance-of-lust"]
]

for book in books:
    meaningful_words = meaningful_words_counter(book)
    print(book)
    meaningful_words.show();

books/frankenstein.txt
+----------+-----+
|meaningful|count|
+----------+-----+
|       one|  204|
|       yet|  152|
|       man|  136|
|    father|  134|
|      upon|  128|
|      life|  116|
|       may|  114|
|     every|  109|
|     first|  108|
|     might|  108|
|     shall|  107|
|      eyes|  104|
|      said|  102|
|      time|   98|
|      even|   96|
|       saw|   94|
|   towards|   94|
| elizabeth|   91|
|     found|   90|
|     night|   88|
+----------+-----+
only showing top 20 rows

books/moby-dick.txt
+----------+-----+
|meaningful|count|
+----------+-----+
|     whale| 1238|
|       one|  916|
|      like|  576|
|      upon|  567|
|       man|  512|
|      ahab|  509|
|      ship|  481|
|       old|  450|
|       sea|  450|
|        ye|  435|
|    though|  369|
|       yet|  340|
|      long|  329|
|      time|  326|
|   captain|  322|
|     still|  308|
|   chapter|  308|
|     great|  307|
|      boat|  304|
|      said|  304|
+----------+-----+
only showing top 20